In [1]:
#!cat ~/.bashrc

In [2]:
#!mv ../drone_dataset.pkl .

In [3]:
#!pip3 install --upgrade protobuf==3.20.0 

In [4]:
#!pip3 install transformers==4.5.1
#!pip3 install -U tokenizers
# The code below just solve many problems lol
#!pip3 uninstall tokenizers -y

In [5]:
from torch.utils.tensorboard import SummaryWriter
import argparse
import pickle
import random
import time
import gym
import d4rl
import torch
import numpy as np

import utils
from replay_buffer import ReplayBuffer
from lamb import Lamb
from stable_baselines3.common.vec_env import SubprocVecEnv
from pathlib import Path
from data import create_dataloader
from decision_transformer.models.decision_transformer import DecisionTransformer
from evaluation import create_vec_eval_episodes_fn, vec_evaluate_episode_rtg
from trainer import SequenceTrainer
from logger import Logger

from env import make_pytorch_env

MAX_EPISODE_LEN = 4000 # 4000 # Warning: there is a similar variable in data.py! 

/home/gabriel/CARLA_0.9.8/PythonAPI/carla/dist/carla-0.9.8-py3.5-linux-x86_64.egg/carla/libcarla.py:3: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
pybullet build time: May 20 2022 19:44:17


In [6]:
import sys
sys.argv = ['']

parser = argparse.ArgumentParser()
parser.add_argument("--seed", type=int, default=10)
parser.add_argument("--env", type=str, default="drone_dataset")
#parser.add_argument("--env", type=str, default="antmaze-large-diverse-v2")

# model options
#parser.add_argument("--K", type=int, default=20)
#parser.add_argument("--K", type=int, default=200)
#parser.add_argument("--K", type=int, default=80)
parser.add_argument("--K", type=int, default=40)
parser.add_argument("--embed_dim", type=int, default=512)
parser.add_argument("--n_layer", type=int, default=4)
parser.add_argument("--n_head", type=int, default=4)
parser.add_argument("--activation_function", type=str, default="relu")
parser.add_argument("--dropout", type=float, default=0.1)
#parser.add_argument("--eval_context_length", type=int, default=5)
#parser.add_argument("--eval_context_length", type=int, default=50)
#parser.add_argument("--eval_context_length", type=int, default=20)
parser.add_argument("--eval_context_length", type=int, default=10)
# 0: no pos embedding others: absolute ordering
#parser.add_argument("--ordering", type=int, default=0)
parser.add_argument("--ordering", type=int, default=1)

# shared evaluation options
parser.add_argument("--eval_rtg", type=int, default=3600)
parser.add_argument("--num_eval_episodes", type=int, default=10)

# shared training options
parser.add_argument("--init_temperature", type=float, default=0.1)
#parser.add_argument("--batch_size", type=int, default=256)
#parser.add_argument("--batch_size", type=int, default=32)
parser.add_argument("--batch_size", type=int, default=128)
parser.add_argument("--learning_rate", "-lr", type=float, default=1e-4)
parser.add_argument("--weight_decay", "-wd", type=float, default=5e-4)
parser.add_argument("--warmup_steps", type=int, default=10000)

# pretraining options
parser.add_argument("--max_pretrain_iters", type=int, default=1)
parser.add_argument("--num_updates_per_pretrain_iter", type=int, default=5000)

# finetuning options
parser.add_argument("--max_online_iters", type=int, default=1500)
parser.add_argument("--online_rtg", type=int, default=7200)
parser.add_argument("--num_online_rollouts", type=int, default=1)
parser.add_argument("--replay_size", type=int, default=1000)
parser.add_argument("--num_updates_per_online_iter", type=int, default=300)
parser.add_argument("--eval_interval", type=int, default=10)

# environment options
parser.add_argument("--device", type=str, default="cuda")
parser.add_argument("--log_to_tb", "-w", type=bool, default=True)
parser.add_argument("--save_dir", type=str, default="./exp")
parser.add_argument("--exp_name", type=str, default="default")

args = parser.parse_args()

In [7]:
class Experiment:
    def __init__(self, variant):

        self.state_dim, self.act_dim, self.action_range = self._get_env_spec(variant)
        self.offline_trajs, self.state_mean, self.state_std = self._load_dataset(
            variant["env"]
        )
        # initialize by offline trajs
        self.replay_buffer = ReplayBuffer(variant["replay_size"], self.offline_trajs)

        self.aug_trajs = []

        self.device = variant.get("device", "cuda")
        self.target_entropy = -self.act_dim
        self.model = DecisionTransformer(
            state_dim=self.state_dim,
            act_dim=self.act_dim,
            action_range=self.action_range,
            max_length=variant["K"],
            eval_context_length=variant["eval_context_length"],
            max_ep_len=MAX_EPISODE_LEN,
            hidden_size=variant["embed_dim"],
            n_layer=variant["n_layer"],
            n_head=variant["n_head"],
            n_inner=4 * variant["embed_dim"],
            activation_function=variant["activation_function"],
            n_positions=1024,
            resid_pdrop=variant["dropout"],
            attn_pdrop=variant["dropout"],
            stochastic_policy=True,
            ordering=variant["ordering"],
            init_temperature=variant["init_temperature"],
            target_entropy=self.target_entropy,
        ).to(device=self.device)

        self.optimizer = Lamb(
            self.model.parameters(),
            lr=variant["learning_rate"],
            weight_decay=variant["weight_decay"],
            eps=1e-8,
        )
        self.scheduler = torch.optim.lr_scheduler.LambdaLR(
            self.optimizer, lambda steps: min((steps + 1) / variant["warmup_steps"], 1)
        )

        self.log_temperature_optimizer = torch.optim.Adam(
            [self.model.log_temperature],
            lr=1e-4,
            betas=[0.9, 0.999],
        )

        # track the training progress and
        # training/evaluation/online performance in all the iterations
        self.pretrain_iter = 0
        self.online_iter = 0
        self.total_transitions_sampled = 0
        self.variant = variant
        self.reward_scale = 1.0 if "antmaze" in variant["env"] else 0.001
        self.logger = Logger(variant)

    def _get_env_spec(self, variant):
        #####env = gym.make(variant["env"])
        env = make_pytorch_env(args)
        env.max_step = MAX_EPISODE_LEN
        state_dim = env.observation_space.shape[0]
        act_dim = env.action_space.shape[0]
        #action_range = [-0.999999, 0.999999]
        
        action_range = [
            float(env.action_space.low.min()) + 1e-6,
            float(env.action_space.high.max()) - 1e-6,
        ]
        
        print("action_range: {}".format(action_range))
        env.close()
        return state_dim, act_dim, action_range

    def _save_model(self, path_prefix, is_pretrain_model=False):
        to_save = {
            "model_state_dict": self.model.state_dict(),
            "optimizer_state_dict": self.optimizer.state_dict(),
            "scheduler_state_dict": self.scheduler.state_dict(),
            "pretrain_iter": self.pretrain_iter,
            "online_iter": self.online_iter,
            "args": self.variant,
            "total_transitions_sampled": self.total_transitions_sampled,
            "np": np.random.get_state(),
            "python": random.getstate(),
            "pytorch": torch.get_rng_state(),
            "log_temperature_optimizer_state_dict": self.log_temperature_optimizer.state_dict(),
        }

        with open(f"{path_prefix}/model.pt", "wb") as f:
            torch.save(to_save, f)
        print(f"\nModel saved at {path_prefix}/model.pt")

        if is_pretrain_model:
            with open(f"{path_prefix}/pretrain_model.pt", "wb") as f:
                torch.save(to_save, f)
            print(f"Model saved at {path_prefix}/pretrain_model.pt")

    def _load_model(self, path_prefix):
        if Path(f"{path_prefix}/model.pt").exists():
            with open(f"{path_prefix}/model.pt", "rb") as f:
                checkpoint = torch.load(f)
            self.model.load_state_dict(checkpoint["model_state_dict"])
            self.optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
            self.scheduler.load_state_dict(checkpoint["scheduler_state_dict"])
            self.log_temperature_optimizer.load_state_dict(
                checkpoint["log_temperature_optimizer_state_dict"]
            )
            self.pretrain_iter = checkpoint["pretrain_iter"]
            self.online_iter = checkpoint["online_iter"]
            self.total_transitions_sampled = checkpoint["total_transitions_sampled"]
            np.random.set_state(checkpoint["np"])
            random.setstate(checkpoint["python"])
            torch.set_rng_state(checkpoint["pytorch"])
            print(f"Model loaded at {path_prefix}/model.pt")

    def _load_dataset(self, env_name):

        dataset_path = f"./data/{env_name}.pkl"
        with open(dataset_path, "rb") as f:
            trajectories = pickle.load(f)

        states, traj_lens, returns = [], [], []
        for path in trajectories:
            states.append(path["observations"])
            traj_lens.append(len(path["observations"]))
            returns.append(path["rewards"].sum())
        traj_lens, returns = np.array(traj_lens), np.array(returns)

        # used for input normalization
        states = np.concatenate(states, axis=0)
        state_mean, state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6
        num_timesteps = sum(traj_lens)

        print("=" * 50)
        print(f"Starting new experiment: {env_name}")
        print(f"{len(traj_lens)} trajectories, {num_timesteps} timesteps found")
        print(f"Average return: {np.mean(returns):.2f}, std: {np.std(returns):.2f}")
        print(f"Max return: {np.max(returns):.2f}, min: {np.min(returns):.2f}")
        print(f"Average length: {np.mean(traj_lens):.2f}, std: {np.std(traj_lens):.2f}")
        print(f"Max length: {np.max(traj_lens):.2f}, min: {np.min(traj_lens):.2f}")
        print("=" * 50)

        sorted_inds = np.argsort(returns)  # lowest to highest
        num_trajectories = 1
        timesteps = traj_lens[sorted_inds[-1]]
        ind = len(trajectories) - 2
        while ind >= 0 and timesteps + traj_lens[sorted_inds[ind]] < num_timesteps:
            timesteps += traj_lens[sorted_inds[ind]]
            num_trajectories += 1
            ind -= 1
        sorted_inds = sorted_inds[-num_trajectories:]
        trajectories = [trajectories[ii] for ii in sorted_inds]

        return trajectories, state_mean, state_std

    def _augment_trajectories(
        self,
        online_envs,
        target_explore,
        n,
        randomized=False,
    ):

        max_ep_len = MAX_EPISODE_LEN

        with torch.no_grad():
            # generate init state
            target_return = [target_explore * self.reward_scale] * online_envs.num_envs

            returns, lengths, trajs = vec_evaluate_episode_rtg(
                online_envs,
                self.state_dim,
                self.act_dim,
                self.model,
                max_ep_len=max_ep_len,
                reward_scale=self.reward_scale,
                target_return=target_return,
                mode="normal",
                state_mean=self.state_mean,
                state_std=self.state_std,
                device=self.device,
                use_mean=False,
            )

        self.replay_buffer.add_new_trajs(trajs)
        self.aug_trajs += trajs
        self.total_transitions_sampled += np.sum(lengths)

        return {
            "aug_traj/return": np.mean(returns),
            "aug_traj/length": np.mean(lengths),
        }

    def pretrain(self, eval_envs, loss_fn):
        print("\n\n\n*** Pretrain ***")
        print("----------------")
        print("eval_envs: {}".format(eval_envs))
        print("loss_fn: {}".format(loss_fn))
        
        eval_fns = [
            create_vec_eval_episodes_fn(
                vec_env=eval_envs,
                eval_rtg=self.variant["eval_rtg"],
                state_dim=self.state_dim,
                act_dim=self.act_dim,
                state_mean=self.state_mean,
                state_std=self.state_std,
                device=self.device,
                use_mean=True,
                reward_scale=self.reward_scale,
            )
        ]

        trainer = SequenceTrainer(
            model=self.model,
            optimizer=self.optimizer,
            log_temperature_optimizer=self.log_temperature_optimizer,
            scheduler=self.scheduler,
            device=self.device,
        )

        writer = (
            SummaryWriter(self.logger.log_path) if self.variant["log_to_tb"] else None
        )
        while self.pretrain_iter < self.variant["max_pretrain_iters"]:
            # in every iteration, prepare the data loader
            dataloader = create_dataloader(
                trajectories=self.offline_trajs,
                num_iters=self.variant["num_updates_per_pretrain_iter"],
                batch_size=self.variant["batch_size"],
                max_len=self.variant["K"],
                state_dim=self.state_dim,
                act_dim=self.act_dim,
                state_mean=self.state_mean,
                state_std=self.state_std,
                reward_scale=self.reward_scale,
                action_range=self.action_range,
            )

            train_outputs = trainer.train_iteration(
                loss_fn=loss_fn,
                dataloader=dataloader,
            )
            eval_outputs, eval_reward = self.evaluate(eval_fns)
            outputs = {"time/total": time.time() - self.start_time}
            outputs.update(train_outputs)
            outputs.update(eval_outputs)
            self.logger.log_metrics(
                outputs,
                iter_num=self.pretrain_iter,
                total_transitions_sampled=self.total_transitions_sampled,
                writer=writer,
            )

            self._save_model(
                path_prefix=self.logger.log_path,
                is_pretrain_model=True,
            )

            self.pretrain_iter += 1

    def evaluate(self, eval_fns):
        eval_start = time.time()
        self.model.eval()
        outputs = {}
        for eval_fn in eval_fns:
            o = eval_fn(self.model)
            outputs.update(o)
        outputs["time/evaluation"] = time.time() - eval_start

        eval_reward = outputs["evaluation/return_mean_gm"]
        return outputs, eval_reward

    def online_tuning(self, online_envs, eval_envs, loss_fn):

        print("\n\n\n*** Online Finetuning ***")

        trainer = SequenceTrainer(
            model=self.model,
            optimizer=self.optimizer,
            log_temperature_optimizer=self.log_temperature_optimizer,
            scheduler=self.scheduler,
            device=self.device,
        )
        eval_fns = [
            create_vec_eval_episodes_fn(
                vec_env=eval_envs,
                eval_rtg=self.variant["eval_rtg"],
                state_dim=self.state_dim,
                act_dim=self.act_dim,
                state_mean=self.state_mean,
                state_std=self.state_std,
                device=self.device,
                use_mean=True,
                reward_scale=self.reward_scale,
            )
        ]
        writer = (
            SummaryWriter(self.logger.log_path) if self.variant["log_to_tb"] else None
        )
        while self.online_iter < self.variant["max_online_iters"]:

            outputs = {}
            augment_outputs = self._augment_trajectories(
                online_envs,
                self.variant["online_rtg"],
                n=self.variant["num_online_rollouts"],
            )
            outputs.update(augment_outputs)

            dataloader = create_dataloader(
                trajectories=self.replay_buffer.trajectories,
                num_iters=self.variant["num_updates_per_online_iter"],
                batch_size=self.variant["batch_size"],
                max_len=self.variant["K"],
                state_dim=self.state_dim,
                act_dim=self.act_dim,
                state_mean=self.state_mean,
                state_std=self.state_std,
                reward_scale=self.reward_scale,
                action_range=self.action_range,
            )

            # finetuning
            is_last_iter = self.online_iter == self.variant["max_online_iters"] - 1
            if (self.online_iter + 1) % self.variant[
                "eval_interval"
            ] == 0 or is_last_iter:
                evaluation = True
            else:
                evaluation = False

            train_outputs = trainer.train_iteration(
                loss_fn=loss_fn,
                dataloader=dataloader,
            )
            outputs.update(train_outputs)

            if evaluation:
                eval_outputs, eval_reward = self.evaluate(eval_fns)
                outputs.update(eval_outputs)

            outputs["time/total"] = time.time() - self.start_time

            # log the metrics
            self.logger.log_metrics(
                outputs,
                iter_num=self.pretrain_iter + self.online_iter,
                total_transitions_sampled=self.total_transitions_sampled,
                writer=writer,
            )

            self._save_model(
                path_prefix=self.logger.log_path,
                is_pretrain_model=False,
            )

            self.online_iter += 1

    def __call__(self):

        utils.set_seed_everywhere(args.seed)

        import d4rl

        def loss_fn(
            a_hat_dist,     # action_preds
            a,              # action_target
            attention_mask, # padding_mask
            entropy_reg,    # self.model.temperature().detach()
        ):
            # a_hat is a SquashedNormal Distribution
            log_likelihood = a_hat_dist.log_likelihood(a)[attention_mask > 0].mean()
            
            entropy = a_hat_dist.entropy().mean()
            loss = -(log_likelihood + entropy_reg * entropy)
            
            '''
            print("a_hat_dist : {}".format(a_hat_dist))
            print("a : {}".format(a))
            torch.save(a,"a.pt")
            print("a_hat_dist.log_likelihood(a) : {}".format(a_hat_dist.log_likelihood(a)))
            #print("attention_mask : {}".format(attention_mask))
            print("log_likelihood: {}".format(log_likelihood))
            print("loss inside jupyter: {} of type: {}".format(loss,type(loss)))
            '''
            
            return (
                loss,
                -log_likelihood,
                entropy,
            )

        def get_env_builder(seed, env_name, target_goal=None):
            def make_env_fn():
                import d4rl

                #####env = gym.make(env_name)
                env = make_pytorch_env(args)
                env.max_step = MAX_EPISODE_LEN
                env.seed(seed)
                '''
                if hasattr(env.env, "wrapped_env"):
                    env.env.wrapped_env.seed(seed)
                elif hasattr(env.env, "seed"):
                    env.env.seed(seed)
                else:
                    pass
                '''
                '''
                env.action_space.seed(seed)
                env.observation_space.seed(seed)
                '''

                if target_goal:
                    env.set_target_goal(target_goal)
                    print(f"Set the target goal to be {env.target_goal}")
                return env

            return make_env_fn

        print("\n\nMaking Eval Env.....")
        env_name = self.variant["env"]
        if "antmaze" in env_name:
            env = gym.make(env_name)
            target_goal = env.target_goal
            env.close()
            print(f"Generated the fixed target goal: {target_goal}")
        else:
            target_goal = None
        eval_envs = SubprocVecEnv(
            [
                get_env_builder(i, env_name=env_name, target_goal=target_goal)
                for i in range(self.variant["num_eval_episodes"])
            ]
        )

        self.start_time = time.time()
        if self.variant["max_pretrain_iters"]:
            self.pretrain(eval_envs, loss_fn)

        if self.variant["max_online_iters"]:
            print("\n\nMaking Online Env.....")
            online_envs = SubprocVecEnv(
                [
                    get_env_builder(i + 100, env_name=env_name, target_goal=target_goal)
                    for i in range(self.variant["num_online_rollouts"])
                ]
            )
            self.online_tuning(online_envs, eval_envs, loss_fn)
            online_envs.close()

        eval_envs.close()

In [ ]:
utils.set_seed_everywhere(args.seed)
experiment = Experiment(vars(args))

print("=" * 50)
experiment()

/home/gabriel/miniconda3/envs/odt/lib/python3.8/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


action_range: [-0.999999, 0.999999]
Starting new experiment: drone_dataset
1254 trajectories, 1971662 timesteps found
Average return: 3687.11, std: 875.27
Max return: 5216.00, min: 1264.00
Average length: 1572.30, std: 325.37
Max length: 2000.00, min: 920.00
Experiment log path: ./exp/2023.03.22/192943-default


Making Eval Env.....



*** Pretrain ***
----------------
eval_envs: <stable_baselines3.common.vec_env.subproc_vec_env.SubprocVecEnv object at 0x7f6425ea2d30>
loss_fn: <function Experiment.__call__.<locals>.loss_fn at 0x7f6425dd7ee0>


pybullet build time: May 20 2022 19:44:17
pybullet build time: May 20 2022 19:44:17
pybullet build time: May 20 2022 19:44:17
pybullet build time: May 20 2022 19:44:17
pybullet build time: May 20 2022 19:44:17
pybullet build time: May 20 2022 19:44:17
pybullet build time: May 20 2022 19:44:17
pybullet build time: May 20 2022 19:44:17
pybullet build time: May 20 2022 19:44:17
pybullet build time: May 20 2022 19:44:17


Iteration 0
time/total: 662.2079253196716
time/training: 644.9144990444183
training/train_loss_mean: 2548.5900681017292
training/train_loss_std: 7701.98695884238
training/nll: -7.420923233032227
training/entropy: -4.700595378875732
training/temp_value: 0.13960246423171133
evaluation/return_mean_gm: -26911.09551140177
evaluation/return_std_gm: 87.56313613335602
evaluation/length_mean_gm: 4000.0
evaluation/length_std_gm: 0.0
time/evaluation: 17.25350594520569

Model saved at ./exp/2023.03.22/192943-default/model.pt
Model saved at ./exp/2023.03.22/192943-default/pretrain_model.pt


Making Online Env.....


pybullet build time: May 20 2022 19:44:17


...


Model saved at ./exp/2023.03.22/192943-default/model.pt
Iteration 222
aug_traj/return: -6859.23398873659
aug_traj/length: 4000.0
time/training: 37.33485412597656
training/train_loss_mean: -3.590819630994594
training/train_loss_std: 0.26776906881435875
training/nll: -6.9165358543396
training/entropy: -3.1829066276550293
training/temp_value: 1.0077079264418625
time/total: 11484.543875932693

Model saved at ./exp/2023.03.22/192943-default/model.pt
Iteration 223
aug_traj/return: -7363.4813905446945
aug_traj/length: 4000.0
time/training: 38.36253786087036
training/train_loss_mean: -3.5833591108938343
training/train_loss_std: 0.2661305205563745
training/nll: -6.675202369689941
training/entropy: -3.0970382690429688
training/temp_value: 1.0061350371794293
time/total: 11531.317291736603

Model saved at ./exp/2023.03.22/192943-default/model.pt
Iteration 224
aug_traj/return: -5086.041847908027
aug_traj/length: 4000.0
time/training: 38.4063823223114
training/train_loss_mean: -3.574360403227093
tr


Model saved at ./exp/2023.03.22/192943-default/model.pt
Iteration 239
aug_traj/return: -4336.942279667521
aug_traj/length: 4000.0
time/training: 37.982481718063354
training/train_loss_mean: -3.476332429005523
training/train_loss_std: 0.25261614686483264
training/nll: -6.375866889953613
training/entropy: -3.028280019760132
training/temp_value: 1.0042928351027596
time/total: 12297.622365951538

Model saved at ./exp/2023.03.22/192943-default/model.pt
Iteration 240
aug_traj/return: -9072.919414385698
aug_traj/length: 4000.0
time/training: 37.57859778404236
training/train_loss_mean: -3.490267972948433
training/train_loss_std: 0.25458313274772776
training/nll: -6.277682781219482
training/entropy: -2.709136962890625
training/temp_value: 1.0048461632637238
evaluation/return_mean_gm: -25890.24265555714
evaluation/return_std_gm: 1234.9704723038988
evaluation/length_mean_gm: 4000.0
evaluation/length_std_gm: 0.0
time/evaluation: 16.188983917236328
time/total: 12360.069968223572

Model saved at ./


Model saved at ./exp/2023.03.22/192943-default/model.pt
Iteration 256
aug_traj/return: -5303.811026280224
aug_traj/length: 4000.0
time/training: 38.678760290145874
training/train_loss_mean: -3.413938159733763
training/train_loss_std: 0.2642750064347933
training/nll: -6.204018592834473
training/entropy: -2.9363105297088623
training/temp_value: 1.002981197134552
time/total: 13131.36930680275

Model saved at ./exp/2023.03.22/192943-default/model.pt
Iteration 257
aug_traj/return: -7173.1987902675355
aug_traj/length: 4000.0
time/training: 37.74834108352661
training/train_loss_mean: -3.387562664389871
training/train_loss_std: 0.2606723182074221
training/nll: -6.619773864746094
training/entropy: -3.0883028507232666
training/temp_value: 1.00350673554083
time/total: 13177.660103321075

Model saved at ./exp/2023.03.22/192943-default/model.pt
Iteration 258
aug_traj/return: -6640.173188318894
aug_traj/length: 4000.0
time/training: 38.17473793029785
training/train_loss_mean: -3.398057842949711
tra


Model saved at ./exp/2023.03.22/192943-default/model.pt
Iteration 273
aug_traj/return: -3852.0454450778493
aug_traj/length: 4000.0
time/training: 38.00166153907776
training/train_loss_mean: -3.2989287042206783
training/train_loss_std: 0.24760297847639143
training/nll: -6.356984615325928
training/entropy: -3.127495050430298
training/temp_value: 1.000440363538643
time/total: 13968.433626413345

Model saved at ./exp/2023.03.22/192943-default/model.pt
Iteration 274
aug_traj/return: -3971.027473068078
aug_traj/length: 4000.0
time/training: 38.420623540878296
training/train_loss_mean: -3.3030205953034653
training/train_loss_std: 0.25701370608980084
training/nll: -6.646392345428467
training/entropy: -3.202181339263916
training/temp_value: 1.0000649209206323
time/total: 14015.293743133545

Model saved at ./exp/2023.03.22/192943-default/model.pt
Iteration 275
aug_traj/return: -4085.059780938862
aug_traj/length: 4000.0
time/training: 37.60509634017944
training/train_loss_mean: -3.33147699533265


Model saved at ./exp/2023.03.22/192943-default/model.pt
Iteration 290
aug_traj/return: -6907.81670717922
aug_traj/length: 4000.0
time/training: 37.827624559402466
training/train_loss_mean: -3.255774269680486
training/train_loss_std: 0.25989332870546206
training/nll: -6.453128814697266
training/entropy: -3.0915896892547607
training/temp_value: 1.0002174532352668
evaluation/return_mean_gm: -29147.938595767337
evaluation/return_std_gm: 378.6997290772935
evaluation/length_mean_gm: 4000.0
evaluation/length_std_gm: 0.0
time/evaluation: 16.209022521972656
time/total: 14801.730618715286

Model saved at ./exp/2023.03.22/192943-default/model.pt
Iteration 291
aug_traj/return: -5488.050614504726
aug_traj/length: 4000.0
time/training: 38.73582220077515
training/train_loss_mean: -3.2289623868247
training/train_loss_std: 0.28096445151157995
training/nll: -6.745492458343506
training/entropy: -3.2328507900238037
training/temp_value: 0.9994670285856191
time/total: 14848.921913385391

Model saved at ./e

In [ ]:
def study_env(env):
    
    state_dim = env.observation_space.shape[0]
    act_dim = env.action_space.shape[0]
    action_range = [
        float(env.action_space.low.min()) + 1e-6,
        float(env.action_space.high.max()) - 1e-6]
        
    print("state_dim: {}".format(state_dim))
    print("act_dim: {}".format(act_dim))
    print("action_range: {}".format(action_range))


In [ ]:
my_env = make_pytorch_env(args)
their_env = gym.make('antmaze-large-diverse-v2')

In [ ]:
study_env(my_env)

In [ ]:
study_env(their_env)

In [ ]:
my_env.reset()
my_env.step(2)

In [ ]:
args

In [ ]:
their_env.action_space

In [ ]:
their_env.reset()
their_env.step(2)

In [ ]:
#experiment.variant
#experiment.model.forward

In [ ]:
loss

In [ ]:
experiment.model.forward

In [ ]:
experiment.model

In [ ]:
import math
math.log(1e-310)

In [ ]:
action_preds = torch.load('action_preds.pt')


In [ ]:
a = torch.load("a.pt")

In [ ]:
action_preds.log_likelihood(a)

In [ ]:
sefude = action_preds.log_likelihood(a)

In [ ]:
a

In [ ]:
a

In [ ]:
a

In [ ]:
torch.nan_to_num(sefude)

In [ ]:
action_preds

In [ ]:
a[0][0]

In [ ]:
math.log(-0.3)

In [ ]:
action_preds.entropy().mean()

In [ ]:
action_preds.log_likelihood(10)

In [ ]:
action_preds.perplexity

In [ ]:
import torch
state_dim = 4
hidden_size = 512

embed_state = torch.nn.Linear(state_dim, hidden_size).to('cuda')
embed_state_2 = torch.load('embed_state.pt').to('cuda')
states = torch.load('states.pt').to('cuda')
state_embeddings = embed_state(states)
state_embeddings_2 = torch.load('state_embeddings.pt').to('cuda')


In [ ]:
states[0]

In [ ]:
print("state_embeddings {}".format(state_embeddings))


In [ ]:
print("state_embeddings 2 {}".format(state_embeddings_2))


In [ ]:
embed_state.weight

In [ ]:
embed_state_2.weight

In [ ]:
embed_state

In [ ]:
embed_state_2

In [ ]:
embed_state(states)

In [ ]:
embed_state_2(states)

In [ ]:
stoppppppppppp

In [ ]:
import torch
torch.__version__

In [ ]:
!pip list | grep torch

In [ ]:
!pip3 install torch --upgrade

In [ ]:
# Normalizando as rewards pra ver se resolve o problema

In [ ]:
import pickle

with open('data/drone_dataset.pkl', 'rb') as f:
    my_data = pickle.load(f)
    
with open('data/antmaze-large-diverse-v2.pkl', 'rb') as f:
    their_data = pickle.load(f)

In [ ]:
for data in my_data:
    rewards = data['actions']
    print("max: {}".format(np.max(rewards)))
    print("min: {}".format(np.min(rewards)))
    print("mean: {}".format(np.mean(rewards)))
    print('----------------')

In [ ]:
np.shape(my_data[0]['observations'])

In [ ]:
np.shape(their_data[0]['observations'])

In [ ]:
(v - v.min()) / (v.max() - v.min())